# Sentiment Analysis

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from bs4 import BeautifulSoup
import re
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')

In [5]:
data = pd.read_csv('labeledTrainData.tsv', delimiter='\t', quoting=3)
data

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."
...,...,...,...
24995,"""3453_3""",0,"""It seems like more consideration has gone int..."
24996,"""5064_1""",0,"""I don't believe they made this film. Complete..."
24997,"""10905_3""",0,"""Guy is a loser. Can't get girls, needs to bui..."
24998,"""10194_3""",0,"""This 30 minute documentary Buñuel made in the..."


In [21]:
test_data = pd.read_csv('testData.tsv', delimiter='\t', quoting=3)
test_data

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."
...,...,...
24995,"""2155_10""","""Sony Pictures Classics, I'm looking at you! S..."
24996,"""59_10""","""I always felt that Ms. Merkerson had never go..."
24997,"""2531_1""","""I was so disappointed in this movie. I am ver..."
24998,"""7772_8""","""From the opening sequence, filled with black ..."


In [7]:
len(data)

25000

In [8]:
len(data['review'])

25000

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Roshan
[nltk_data]     Upadhyay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
sample = data.review[0]
sample

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [11]:
# Clear HTML tags
sample = BeautifulSoup(sample).get_text()
sample

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 2

In [12]:
# removing punctuation and numberssample = re.sub("[^a-zA-Z]",' ',sample)
sample = re.sub("[^a-zA-Z]",' ',sample)
sample

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    m

In [13]:
# converting to lower case
sample = sample.lower()
sample

' with all this stuff going down at the moment with mj i ve started listening to his music  watching the odd documentary here and there  watched the wiz and watched moonwalker again  maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  some of it has subtle messages about mj s feeling towards the press and also the obvious message of drugs are bad m kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring  some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for    m

In [14]:
# tokenization
sample = sample.split()
sample

['with',
 'all',
 'this',
 'stuff',
 'going',
 'down',
 'at',
 'the',
 'moment',
 'with',
 'mj',
 'i',
 've',
 'started',
 'listening',
 'to',
 'his',
 'music',
 'watching',
 'the',
 'odd',
 'documentary',
 'here',
 'and',
 'there',
 'watched',
 'the',
 'wiz',
 'and',
 'watched',
 'moonwalker',
 'again',
 'maybe',
 'i',
 'just',
 'want',
 'to',
 'get',
 'a',
 'certain',
 'insight',
 'into',
 'this',
 'guy',
 'who',
 'i',
 'thought',
 'was',
 'really',
 'cool',
 'in',
 'the',
 'eighties',
 'just',
 'to',
 'maybe',
 'make',
 'up',
 'my',
 'mind',
 'whether',
 'he',
 'is',
 'guilty',
 'or',
 'innocent',
 'moonwalker',
 'is',
 'part',
 'biography',
 'part',
 'feature',
 'film',
 'which',
 'i',
 'remember',
 'going',
 'to',
 'see',
 'at',
 'the',
 'cinema',
 'when',
 'it',
 'was',
 'originally',
 'released',
 'some',
 'of',
 'it',
 'has',
 'subtle',
 'messages',
 'about',
 'mj',
 's',
 'feeling',
 'towards',
 'the',
 'press',
 'and',
 'also',
 'the',
 'obvious',
 'message',
 'of',
 'drugs',

In [15]:
len(sample)

437

In [16]:
swords = set(stopwords.words('english'))
sample = [x for x in sample if x not in swords]
sample

['stuff',
 'going',
 'moment',
 'mj',
 'started',
 'listening',
 'music',
 'watching',
 'odd',
 'documentary',
 'watched',
 'wiz',
 'watched',
 'moonwalker',
 'maybe',
 'want',
 'get',
 'certain',
 'insight',
 'guy',
 'thought',
 'really',
 'cool',
 'eighties',
 'maybe',
 'make',
 'mind',
 'whether',
 'guilty',
 'innocent',
 'moonwalker',
 'part',
 'biography',
 'part',
 'feature',
 'film',
 'remember',
 'going',
 'see',
 'cinema',
 'originally',
 'released',
 'subtle',
 'messages',
 'mj',
 'feeling',
 'towards',
 'press',
 'also',
 'obvious',
 'message',
 'drugs',
 'bad',
 'kay',
 'visually',
 'impressive',
 'course',
 'michael',
 'jackson',
 'unless',
 'remotely',
 'like',
 'mj',
 'anyway',
 'going',
 'hate',
 'find',
 'boring',
 'may',
 'call',
 'mj',
 'egotist',
 'consenting',
 'making',
 'movie',
 'mj',
 'fans',
 'would',
 'say',
 'made',
 'fans',
 'true',
 'really',
 'nice',
 'actual',
 'feature',
 'film',
 'bit',
 'finally',
 'starts',
 'minutes',
 'excluding',
 'smooth',
 'crim

In [17]:
len(sample)

219

In [18]:
def preprocessing(review):
    review = BeautifulSoup(review).get_text()
    review = re.sub('[^a-zA-Z]',' ',review)
    review = review.lower()
    review = review.split()
    swords = set(stopwords.words("english"))
    review = [w for w in review if w not in swords]
    return (' '.join(review))

In [19]:
train = []
for r in range(len(data["review"])):
    if (r+1)%1000 == 0:
        print('No. of reviews processed = ',r+1)
    train.append(preprocessing(data["review"][r]))

No. of reviews processed =  1000
No. of reviews processed =  2000
No. of reviews processed =  3000
No. of reviews processed =  4000
No. of reviews processed =  5000
No. of reviews processed =  6000
No. of reviews processed =  7000
No. of reviews processed =  8000
No. of reviews processed =  9000
No. of reviews processed =  10000
No. of reviews processed =  11000
No. of reviews processed =  12000
No. of reviews processed =  13000
No. of reviews processed =  14000
No. of reviews processed =  15000
No. of reviews processed =  16000
No. of reviews processed =  17000
No. of reviews processed =  18000
No. of reviews processed =  19000
No. of reviews processed =  20000
No. of reviews processed =  21000
No. of reviews processed =  22000
No. of reviews processed =  23000
No. of reviews processed =  24000
No. of reviews processed =  25000


In [20]:
print(train[0])

stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate working

In [22]:
test = []
for r in range(len(test_data["review"])):
    if (r+1)%1000 == 0:
        print('No. of reviews processed = ',r+1)
    test.append(preprocessing(test_data["review"][r]))

No. of reviews processed =  1000
No. of reviews processed =  2000
No. of reviews processed =  3000
No. of reviews processed =  4000
No. of reviews processed =  5000
No. of reviews processed =  6000
No. of reviews processed =  7000
No. of reviews processed =  8000
No. of reviews processed =  9000
No. of reviews processed =  10000
No. of reviews processed =  11000
No. of reviews processed =  12000
No. of reviews processed =  13000
No. of reviews processed =  14000
No. of reviews processed =  15000
No. of reviews processed =  16000
No. of reviews processed =  17000
No. of reviews processed =  18000
No. of reviews processed =  19000
No. of reviews processed =  20000
No. of reviews processed =  21000
No. of reviews processed =  22000
No. of reviews processed =  23000
No. of reviews processed =  24000
No. of reviews processed =  25000


In [36]:
print(test[0])

naturally film main themes mortality nostalgia loss innocence perhaps surprising rated highly older viewers younger ones however craftsmanship completeness film anyone enjoy pace steady constant characters full engaging relationships interactions natural showing need floods tears show emotion screams show fear shouting show dispute violence show anger naturally joyce short story lends film ready made structure perfect polished diamond small changes huston makes inclusion poem fit neatly truly masterpiece tact subtlety overwhelming beauty


In [23]:
x = train
y = np.array(data['sentiment'])

**Buliding Bag Of Words**

In [24]:
vectorizer = CountVectorizer(max_features = 10000)
vectorizer

CountVectorizer(max_features=10000)

In [25]:
train_x = vectorizer.fit_transform(x)
train_x

<25000x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 2179303 stored elements in Compressed Sparse Row format>

In [26]:
train_x = train_x.toarray()
train_x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [27]:
train_x.shape, y.shape

((25000, 10000), (25000,))

In [28]:
y

array([1, 1, 0, ..., 0, 0, 1], dtype=int64)

In [29]:
model = RandomForestClassifier(n_estimators= 500, random_state=42)
model.fit(train_x, y)

RandomForestClassifier(n_estimators=500, random_state=42)

**Test**

In [48]:
pred = model.predict(train_x)
pred

array([1, 1, 0, ..., 0, 0, 1], dtype=int64)

In [49]:
acc = roc_auc_score(y, pred)

In [50]:
print('Accuracy: ',acc*100)

Accuracy:  100.0


In [51]:
test_xx = vectorizer.fit_transform(test) 

In [52]:
test_xx

<25000x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 2136410 stored elements in Compressed Sparse Row format>

In [53]:
test_xx = test_xx.toarray()
test_xx

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [54]:
test_xx.shape

(25000, 10000)

In [57]:
pred = model.predict(test_xx)

In [59]:
pred

array([1, 1, 0, ..., 1, 0, 0], dtype=int64)

In [58]:
test_data['sentiment'] = pred

In [60]:
test_data.to_csv('Test_Results.csv')